In [15]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('h') # - timedelta(hours=1)
print(f'{current_date=}')

current_date=Timestamp('2025-09-07 20:00:00')


In [18]:
# # from src.feature_store_api import load_batch_of_features_from_store

# features = load_batch_of_features_from_store(current_date,n_features=5)

In [19]:
from src.feature_store_api import load_batch_of_features_from_store
from src import config
from src.config import FEATURE_VIEW_METADATA
features = load_batch_of_features_from_store(
    feature_view_metadata=config.FEATURE_VIEW_METADATA,
    n_features=5
)

print(features.head())

2025-09-08 01:45:17,837 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-09-08 01:45:17,842 INFO: Initializing external client
2025-09-08 01:45:17,843 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-09-08 01:45:21,260 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242261
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.68s) 
                pickup_hour  rides  pickup_location_id      pickup_ts
0 2025-08-03 12:00:00+00:00      0                 103  1754222400000
1 2025-08-03 12:00:00+00:00      1                 160  1754222400000
2 2025-08-03 12:00:00+00:00      0                  81  1754222400000
3 2025-08-03 12:00:00+00:00      0                 259  1754222400000
4 2025-08-03 12:00:00+00:00      0                  23  1754222400000


In [23]:
import hopsworks
from src import config

# Login to Hopsworks
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME)

# Project info
print("Connected to Hopsworks project:", project.name)

# Access model registry
mr = project.get_model_registry()

# If you know your model name, fetch it
model_name = "taxi_demand_predictor_next_hour"  # or whatever is in your project
try:
    models = mr.get_models(model_name)
    print(f"Models with name '{model_name}':", models)
except Exception as e:
    print(f"Could not fetch models: {e}")


2025-09-08 00:16:43,566 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-09-08 00:16:43,571 INFO: Initializing external client
2025-09-08 00:16:43,573 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-09-08 00:16:47,741 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242261
Connected to Hopsworks project: taxidemand_predict
Models with name 'taxi_demand_predictor_next_hour': [Model(name: 'taxi_demand_predictor_next_hour', version: 1)]


In [20]:
from src.inference import load_model, get_model_predictions
from src.feature_store_api import load_batch_of_features_from_store
from src import config

# Load features
features = load_batch_of_features_from_store(config.FEATURE_VIEW_METADATA, config.N_FEATURES)

# Load model
model = load_model()

# Generate predictions
predictions = get_model_predictions(model, features)

features['predicted_demand'] = predictions['predicted_demand']
print(features.head())

2025-09-08 01:45:35,660 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-09-08 01:45:35,668 INFO: Initializing external client
2025-09-08 01:45:35,670 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-09-08 01:45:39,063 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242261
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.99s) 
2025-09-08 01:45:54,486 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-09-08 01:45:54,492 INFO: Initializing external client
2025-09-08 01:45:54,493 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-09-08 01:45:57,645 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242261


Downloading: 0.000%|          | 0/129940 elapsed<00:00 remaining<?

❌ Error loading model: [Errno 2] No such file or directory: '/tmp/38ee829b-0141-4951-b0fd-f0d92a192d26/taxi_demand_predictor_next_hour/1/taxi_demand_predictor_next_hour.pkl'
⚠️ Model is None, generating synthetic predictions
                pickup_hour  rides  pickup_location_id      pickup_ts  \
0 2025-07-27 17:00:00+00:00      2                 169  1753635600000   
1 2025-07-27 17:00:00+00:00      1                 134  1753635600000   
2 2025-07-27 17:00:00+00:00      0                 217  1753635600000   
3 2025-07-27 17:00:00+00:00      0                  60  1753635600000   
4 2025-07-27 17:00:00+00:00      0                  80  1753635600000   

   predicted_demand  
0                15  
1                 7  
2                12  
3                 7  
4                 1  


In [21]:
predictions['pickup_hour'] = current_date
predictions

,predicted_demand,pickup_hour
0,15,2025-09-07 20:00:00
1,7,2025-09-07 20:00:00
2,12,2025-09-07 20:00:00
3,7,2025-09-07 20:00:00
4,1,2025-09-07 20:00:00
...,...,...
44515,18,2025-09-07 20:00:00
44516,5,2025-09-07 20:00:00
44517,8,2025-09-07 20:00:00
44518,9,2025-09-07 20:00:00


In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2025-09-08 01:46:38,427 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-09-08 01:46:38,433 INFO: Initializing external client
2025-09-08 01:46:38,434 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-09-08 01:46:41,755 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242261


AttributeError: module 'src.config' has no attribute 'FEATURE_GROUP_MODEL_PREDICTIONS'